In [1]:
code_loc = "/home/shi/WorkSpace/projects/scMultiNet_workspace/"
root = "/home/shi/WorkSpace/projects/scMultiNet_Data/"
save_root = root + "/Step_1_data/Dataset_McF/"
import sys
sys.path.append(code_loc )

In [2]:
# start with a vocab list
# this related with the gene2vec model
#----> pre-trained part
vocab_loc = code_loc +"/support_data/vocab_gene2vec_16906.pkl"
target_label = 'Ground_truth' # the label to be predicted

In [3]:

data_path = root+"/Step_0_data/2023_08_09_EMT_McFaline_vocab_scBERT.h5ad"

In [4]:
# check the data
import scanpy as sc
import pandas as pd
import numpy as np

adata = sc.read(data_path)
print(adata)
print(adata.obs["Ground_truth"].unique())
print(adata.obs["Tissue"].unique())
print(adata.obs["CellLine"].unique())
print(adata.obs["stimulus"].unique())

AnnData object with n_obs × n_vars = 9268 × 16906
    obs: 'Ground_truth', 'n_counts', 'pseudotimes', 'stimulus', 'batch', 'CellLine', 'Tissue', 'Experiment'
    uns: 'log1p', 'source_paper'
    layers: 'X_log1p', 'X_normed'
['Mock_inner', 'Mock_outer', 'TGFB_inner', 'TGFB_outer']
Categories (4, object): ['Mock_inner', 'Mock_outer', 'TGFB_inner', 'TGFB_outer']
['Breast']
Categories (1, object): ['Breast']
['MCF10A']
Categories (1, object): ['MCF10A']
['MOCK', 'TGFb']
Categories (2, object): ['MOCK', 'TGFb']


In [5]:

from scLLM.Dataset.paras import Dataset_para
# define pre-processing by follow original implementation of scBERT

from scLLM.Dataset.paras import Dataset_para
# define pre-processing by follow original implementation of scBERT 
dataset_para_cls = Dataset_para(
                            var_idx=None,
                            obs_idx="Ground_truth",
                            vocab_loc=code_loc + "/Experiment/support_data/vocab_16k.json",
                            filter_gene_by_counts=False,
                            filter_cell_by_counts=200,
                            log1p=True,
                            log1p_base=2,

                            #
                            tokenize_name="scBERT",
                            cls_nb=5,
                            data_layer_name="X_log1p",
                            label_key = target_label,#"Ground_truth",#"Ground_truth",

                            test_size=0.2,#0.2, #use all data to inference
                            binarize=None, # not binarize use original label

                            )

dataset_para_reg = Dataset_para(
        vocab_loc=code_loc +"/Experiment/support_data/vocab_16k.json",
        var_idx = None,#"genes.gene_short_name",
        obs_idx="pseudotimes",
        filter_gene_by_counts=False,
        filter_cell_by_counts=200,
        log1p=True,
        log1p_base=2,

        tokenize_name="scBERT",
        cls_nb=1,
        data_layer_name="X_log1p",

        auto_map_str_labels=False,
        label_key = target_label,#"pseudotimes",

        test_size=0.2,#0.2, #use all data to inference
        binarize=None, # not binarize use original label for regression
    )

dataset_para = dataset_para_cls#dataset_para_cls
print(dataset_para)

Dataset_para(var_idx=None, obs_idx='Ground_truth', vocab_loc='/home/shi/WorkSpace/projects/scMultiNet_workspace//Experiment/support_data/vocab_16k.json', gene_vocab=None, use_key='X', filter_gene_by_counts=False, filter_cell_by_counts=200, normalize_total=10000.0, result_normed_key='X_normed', log1p=True, result_log1p_key='X_log1p', log1p_base=2, subset_hvg=False, hvg_use_key=None, hvg_flavor='seurat_v3', binning=None, result_binned_key='X_binned', tokenize_name='scBERT', return_pt=True, append_cls=True, include_zero_gene=False, cls_token='<cls>', max_len=16000, pad_token='<pad>', pad_value=-2, cls_appended=True, mask_ratio=0.15, mask_value=-1, preprocessed_loc=None, data_layer_name='X_log1p', label_key='Ground_truth', batch_label_key=None, cls_nb=5, binarize=None, bins=None, bin_min=None, bin_max=None, save_in_obs=True, auto_map_str_labels=True, map_dict=None, n_splits=1, test_size=0.2, random_state=2023, shuffle=True, sort_seq_batch=False)


In [ ]:
# if this is the first time to run, need this block to init translate=True
# init preprocessor
from scLLM.Dataset.Reader import scReader
screader = scReader(dataset_para=dataset_para)
# init vocab
screader.init_vocab()

# load data
screader.load_adata(loc = data_path,translate=True)

## preprocess
screader.preprocess()

In [10]:
# if not the first run 
# -----> 读取数据集
from scLLM.Dataset.Reader import scReader
data_reader = scReader(dataset_para)
# init vocab from default file loc or from list/dict given as params
data_reader.init_vocab()
#load anndata
data_reader.load_adata(loc = data_path,translate=False)

scLLM - INFO - Initializing preprocessor ...
scLLM - INFO - use default vocab from dataset_para
scLLM - INFO - load vocab from /home/shi/WorkSpace/projects/scMultiNet_workspace//Experiment/support_data/vocab_16k.json
scLLM - INFO - Load data from anndata object.


In [11]:
target_stimulate = "TGFb"
target_cellline = None #"A549" 'DU145', 'MCF7', 'OVCA420'
map_stimulus = False
map_cancer_type = False

print(f"选择前的数据矩阵形状 {data_reader.adata.shape}")
if map_stimulus:
    stimilus_map = data_reader.adata.obs['stimulus']==target_stimulate
    # filter out cells that are not stimulated by TGFb
    new_adata = data_reader.adata[stimilus_map,:]
    data_reader.adata = new_adata

if map_cancer_type:
    cancer_type_map = data_reader.adata.obs['CellLine']==target_cellline
    # filter out cells that are not stimulated by TGFb
    new_adata = data_reader.adata[cancer_type_map,:]
    data_reader.adata = new_adata

print(f"选择后的数据矩阵形状 {data_reader.adata.shape}")

选择前的数据矩阵形状 (9268, 16906)
选择后的数据矩阵形状 (9268, 16906)


In [13]:
label_unique = data_reader.adata.obs[dataset_para.label_key].unique()
print(label_unique)
label_dict = {'Mock_inner':0, 'Mock_outer':1, 'TGFB_inner':2, 'TGFB_outer':3}

['Mock_inner', 'Mock_outer', 'TGFB_inner', 'TGFB_outer']
Categories (4, object): ['Mock_inner', 'Mock_outer', 'TGFB_inner', 'TGFB_outer']


In [14]:
data_reader.adata.obs[dataset_para.label_key]

AAACATACGAATAG.1_Mock_TGFB_agregated_samples    Mock_inner
AAACATACGCTAAC.1_Mock_TGFB_agregated_samples    Mock_inner
AAACATACGGACAG.1_Mock_TGFB_agregated_samples    Mock_inner
AAACATTGTCTCAT.1_Mock_TGFB_agregated_samples    Mock_inner
AAACCGTGTCACGA.1_Mock_TGFB_agregated_samples    Mock_inner
                                                   ...    
TTTCTACTGGGCAA.4_Mock_TGFB_agregated_samples    TGFB_outer
TTTGACTGAGCCTA.4_Mock_TGFB_agregated_samples    TGFB_outer
TTTGACTGGACGTT.4_Mock_TGFB_agregated_samples    TGFB_outer
TTTGCATGAAGTAG.4_Mock_TGFB_agregated_samples    TGFB_outer
TTTGCATGCTGAGT.4_Mock_TGFB_agregated_samples    TGFB_outer
Name: Ground_truth, Length: 9268, dtype: category
Categories (4, object): ['Mock_inner', 'Mock_outer', 'TGFB_inner', 'TGFB_outer']

In [15]:
trainset,valset,weights = data_reader.postprocess()

# 输出数据集信息
print("trainset size: ",len(trainset))
print("valset size: ",len(valset)) if valset is not None else print("no valset")
print(label_dict)
print("weights: ",weights)

scLLM - INFO - Map string labels to int automatically.
scLLM - INFO - Mapping from {'Mock_inner': 0, 'Mock_outer': 1, 'TGFB_inner': 2, 'TGFB_outer': 3}
scLLM - INFO - Discritize label Ground_truth in obs_names
scLLM - INFO - Classification task, split with StratifiedShuffleSplit


trainset size:  7414
valset size:  1854
{'Mock_inner': 0, 'Mock_outer': 1, 'TGFB_inner': 2, 'TGFB_outer': 3}
weights:  None


In [16]:
import dill
# 为trainset 添加其他labels

#target_task = f"/TrVal_dataset_GT_{target_stimulate}_{target_cellline}.pkl"
target_task = f"/TrVal_dataset_{target_label}.pkl"
loc = save_root + target_task
# 保存 trainset 到文件，并关联相应labels
with open(loc,"wb") as f:
    dill.dump([trainset,valset,weights,label_dict],f)